In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,TimeDistributed,GRU,Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50

In [2]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
for layer in resnet.layers[:-10]:
    layer.trainable=False

In [4]:
model = Sequential([resnet,Flatten(),Dense(64,activation='relu'),Dense(5,activation='softmax')])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                6422592   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 30,010,629
Trainable params: 10,888,581
Non-trainable params: 19,122,048
_________________________________________________________________


In [5]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
cnn =Sequential([resnet])
cnn.add(Conv2D(64,(2,2),strides=(1,1)))
cnn.add(Conv2D(16,(3,3),strides=(1,1)))
cnn.add(Flatten())

In [6]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
conv2d (Conv2D)              (None, 6, 6, 64)          524352    
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 16)          9232      
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
Total params: 24,121,296
Trainable params: 24,068,176
Non-trainable params: 53,120
_________________________________________________________________


In [7]:
model = Sequential()
model.add(TimeDistributed(cnn,input_shape=(30,224,224,3)))
model.add(GRU(16,input_shape=(None,30,256),return_sequences=True))
model.add(GRU(8))
model.add(Dense(5,activation='softmax'))

In [8]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 256)           24121296  
_________________________________________________________________
gru (GRU)                    (None, 30, 16)            13104     
_________________________________________________________________
gru_1 (GRU)                  (None, 8)                 600       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 45        
Total params: 24,135,045
Trainable params: 24,081,925
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
opt = tf.keras.optimizers.SGD(lr=0.001) 
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])